<a href="https://colab.research.google.com/github/TLuckh/DialogActClassifier/blob/main/Dialog_System_Inform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install py7zr

In [ ]:

import numpy as np
import pandas as pd
import numpy as np
import torch, json, os
from torch import nn
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import re
from dataclasses import dataclass
import numpy as np
import pandas as pd
import numpy as np
import torch, json, os
from torch import nn
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import py7zr
from spacy.lang.en import English

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
home = './drive/MyDrive/Colab Notebooks/'

In [ ]:
import zipfile



def unzip(path_to_zip_file, directory_to_extract_to):
  with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
      zip_ref.extractall(directory_to_extract_to)


unzip(home+'glove.6B.100d.zip', '')

In [ ]:
# Loading and organizing the data

@dataclass
class Sample():
    label:  dict
    log:    dict


def load_data(seven_zip_archive_path):
    """
    Reads in the given 7zip-Archive, which is a reformat of the train & test-folders.
    Returns:
        train:  A list of Sample-Classes making up the training set
        test:   A list of Sample-Classes making up the test set
        ontology:  A list of possible slot-types and for each its possible values
    """

    # Load zip and read:
    tmp_file = py7zr.SevenZipFile(seven_zip_archive_path)
    x = tmp_file.readall()
    tmp_file.close()

    #Filter to only obtain .json-Files:
    irrelevant_files = [x for x in list(x.keys()) if "json" not in x]
    relevant_files = [x for x in list(x.keys()) if "json" in x]
    jsons_dict = {t:json.load(x[t]) for t in relevant_files}

    #Remove train/scripts/config/ontology_dstc2.json:
    assert 'train/scripts/config/ontology_dstc2.json' in relevant_files
    relevant_files_new = [x for x in relevant_files if not x == "train/scripts/config/ontology_dstc2.json"]
    ontology  = [x for x in relevant_files if x == "train/scripts/config/ontology_dstc2.json"][0]
    relevant_files = relevant_files_new

    #Split in test & train:
    relevant_training_files = [x for x in relevant_files if 'train' in x]
    relevant_test_files     = [x for x in relevant_files if 'test' in x]

    #Get for each sample its folder containing log.json and label.json:
    sample_folders_train = set()
    sample_folders_test = set()
    for file_name in relevant_training_files:
        prefix = re.findall("^(([^/]+/){4})", file_name)[0]
        sample_folders_train.add(prefix)

    for file_name in relevant_test_files:
        prefix = re.findall("^(([^/]+/){4})", file_name)[0]
        sample_folders_test.add(prefix)
    #Output a tuple (out_train,out_test) such that both are lists of tuples (label.json,log.json)
    out_train = []
    out_test  = []

    for folder_name in sample_folders_train:
        tuple_res   = [x for x in relevant_files if x.startswith(folder_name)]
        label       = [x for x in tuple_res if x.endswith("label.json")][0]
        log         = [x for x in tuple_res if x.endswith("log.json")][0]

        out_train += [Sample(jsons_dict[label],jsons_dict[log])]

    for folder_name in sample_folders_test:
        tuple_res   = [x for x in relevant_files if x.startswith(folder_name)]
        label       = [x for x in tuple_res if x.endswith("label.json")][0]
        log         = [x for x in tuple_res if x.endswith("log.json")][0]

        out_test += [Sample(jsons_dict[label],jsons_dict[log])]

    return (out_train,out_test,ontology)

out_train,out_test, ontology = load_data(home+"Train_Test_Compressed.7z")



In [ ]:
# Loading the gloVe word embeddings
# Due to size restrictions these aren't in the project, but point to an external source; It is written such that all sizes can be used
# GLOVE_LOCATION = r"F:\Temp\NLP\glove.6B.50d.txt"
GLOVE_LOCATION = r"glove.6B.100d.txt"

with open(GLOVE_LOCATION,encoding="UTF-8") as f:
    lines = f.readlines()
    lines = [x.split(" ") for x in lines]
    lines.sort()
    glove_word_count = len(lines)
    glove_vector_length = len(lines[0]) - 1
    word_embedding = {x[0]:np.array(x[1:],dtype=float) for x in lines}
    #Alternative as vector of words, and vector of vectors:
    # words = [x[0] for x in lines]
    # vecs = [np.array(x[1:]) for x in lines]
    # mat = np.stack(vecs)

    


In [ ]:
# A lot of words in Glove are kind of useless... there are for example a lot of numbers in there... see e.g words[:100]

# The words in glvoe seem to be lower-text

def word_to_vector(word_lowercased,embeddings):
    exists = 1
    try:
        embedding = embeddings[word_lowercased]
    except KeyError:
        embedding = np.zeros((glove_vector_length))
        exists = 0
    return (exists,embedding)

# Bisherig:
# out_train,out_test, ontology = load_data("Train_Test_Compressed.7z") ; tupel der trainings- und testdaten (als Klassen-Instanzen von Sample)
#

In [ ]:
class TextDataset(Dataset):
    """
    Input: A list of texts as texts, and in labels for each text whether the pragmatic act of the text is "inform". dim is the word-embedding dimension
    """
    def __init__(self, texts, labels,embeddings):
        self.embeddings = embeddings
        self.texts = []
        self.labels = labels
        self.dim = len(embeddings[list(embeddings.keys())[0]])
        self.vectors = []
        for index, text in enumerate(texts):
            cur_vec = np.zeros((self.dim,),dtype=float)
            existing_embeddings = 0
            for word in text:
                exists, add_vec = self.word_to_vector(word)
                cur_vec += add_vec
                existing_embeddings +=exists
            self.texts += [cur_vec/existing_embeddings]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (self.texts[idx],self.labels[idx])


    # The words in glvoe seem to be lower-text
    def word_to_vector(self, word_lowercased):
        exists = 1
        try:
            embedding = np.array(self.embeddings[word_lowercased],dtype=float)
        except KeyError:
            embedding = np.zeros((self.dim),dtype=float)
            exists = 0
        return (exists,embedding)
        

In [ ]:
#Tests

sentence_test1  = ["this isn't a sentence".split(" "), "this is another sentence".split(" ")]
embeddings_test1 = {"this":[0.1,0.2,0.3],"is":[0,1.,0.3]}

TextDataset(sentence_test1,[1,1],embeddings_test1)[1]

(array([0.05, 0.6 , 0.3 ]), 1)

In [ ]:
# To do: set learning rate
# To do: set number of training epochs
learning_rate = 1e-3
epochs = 10


ACT = 'inform'
TRAIN_FOLDERS = ['Mar13_S0A0', 'Mar13_S0A1', 'Mar13_S1A0', 'Mar13_S1A1']
TEST_FOLDERS = ['Mar13_S2A0', 'Mar13_S2A1']

# To do: set batch size
# To do: set dimension
BATCH_SIZE = 16
DIMENSION = 50
HIDDEN_DIM = 128

In [ ]:
embeddings = {}

# To do: read from the GloVe vectors to create the embeddings dictionary 
# the dictionary maps each word to its representation
with open('./glove.6B/glove.6B.' + str(DIMENSION) + 'd.txt') as handle:
    for line in handle.readlines():
        parts = line.strip('\n').split(' ')
        assert len(parts) == DIMENSION + 1
        embeddings[parts[0]] = [float(i) for i in parts[1:]]

In [ ]:

        
        
def load_data(folder_list):
    # To do: load data
    # the data should be a list of lists of words
    # the labels are a list of labels
    
    all_paths = []
    for folder in folder_list:
        all_paths.extend(['data/' + folder + '/' + i for i in os.listdir('data/' + folder)])

    data = []
    labels = []
    for f in tqdm(all_paths):
        if 'DS_Store' in f:
            continue
        with open(f + '/label.json') as handle:
            label = json.loads(handle.read())
        with open(f + '/log.json') as handle:
            log = json.loads(handle.read())

        assert len(label['turns']) == len(log['turns'])
        for turn in range(len(label['turns'])):
            turn_set = set()
            for act in label['turns'][turn]['semantics']['json']:
                turn_set.add(act['act'])
            labels.append(1 if ACT in turn_set else 0)
            data.append(log['turns'][turn]['input']['live']['asr-hyps'][0]['asr-hyp'].split(' '))

        # print(data)
        # print(labels)

    return TextDataset(data, labels, embeddings, DIMENSION)

In [ ]:
training_data = load_data(TRAIN_FOLDERS)
test_data = load_data(TEST_FOLDERS)

print('Creating dataloaders...')
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # To do: define your network
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(DIMENSION, HIDDEN_DIM),
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, HIDDEN_DIM),
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, 2)
        )

    def forward(self, x):
        # To do: define the forward pass
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork()

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    # To do: write the training loop
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        # print(loss)

        # Backpropagation
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



In [ ]:

def test_loop(dataloader, model, loss_fn):
    # To do: write the test loop
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [ ]:


# To do: define loss function
# To do: define optimizer


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")